## Подключение к Spark

In [ ]:
from pyspark.sql import SparkSession

# Имя хоста Spark Master в docker
SPARK_MASTER_HOST = "spark-master"
# Порт Spark Master
SPARK_MASTER_PORT = "7077"
# Память выделенная для Spark Worker в настройках docker-compose.yml
SPARK_WORKER_MEMORY = "512m"
# Название сессии (любое)
SPARK_SESSION = "pyspark-notebook"

# Создаем сессию
spark = (
    SparkSession.builder.appName(SPARK_SESSION)
    .master(f"spark://{SPARK_MASTER_HOST}:{SPARK_MASTER_PORT}")
    .config("spark.executor.memory", SPARK_WORKER_MEMORY)
    .getOrCreate()
)

## Чтение в Spark из Jupyter workspace

Прочитать файл в Spark можно из рабочей директории Jupyter Lab

In [ ]:
# Директория с данными в workspace Jupyter
BASE_DIR = './data/'
# Название файла
FILENAME = 'test.csv'
# Полное имя файла
file = f"{BASE_DIR}{FILENAME}"

# Чтение файла
spark_df = spark.read.csv(file, header=True)

## Запись в Jupyter workspace

In [ ]:
FILENAME = 'test.csv'

write_jupyter = (
    spark_df.write.option("header", True)
    # .partitionBy("PublishYear")
    .mode("overwrite")
    .parquet(f'{BASE_DIR}{FILENAME}')
)

## Чтение в Spark из Hadoop

In [ ]:
# Имя Hadoop Namenode в docker
HADOOP_HOST = 'hadoop-namenode'
# Hadoop Namenode port
HADOOP_PORT = '9000'
# Название файла
FILENAME = 'test.parquet'

spark_df = spark.read.parquet(f"hdfs://{HADOOP_HOST}:{HADOOP_PORT}/{FILENAME}")

## Запись в Hadoop

In [ ]:
FILENAME = 'test.parquet'

write_hadoop = (
    spark_df.write.option("header", True)
    # .partitionBy("PublishYear")
    .mode("overwrite")
    .parquet(f"hdfs://{HADOOP_HOST}:{HADOOP_PORT}/{FILENAME}")
)

## Создание пустого датафрейма со схемой из csv файла

In [47]:
from pyspark.sql.types import StructType, StructField, StringType

FILENAME = 'test.csv'

with open(FILENAME) as file:
    headers = file.readline().rstrip().split(',')

schema = StructType([StructField(header, StringType(), True) for header in headers])

emp_RDD = spark.sparkContext.emptyRDD()

spark_df = spark.createDataFrame(data = emp_RDD, schema=schema)

## Получение списка файлов в директории по условию

In [ ]:
import os

file_list = [f"{BASE_DIR}{file}" for file in os.listdir(BASE_DIR) if 'test' in file]
file_list

## Загрузка списка файлов с проверкой схемы и ее дополнением при необходимости

In [54]:
for file in file_list:
    spark_data = spark.read.csv(file, header=True)
    spark_df = spark_df.unionByName(spark_data, allowMissingColumns=True)